# Sitemap Check Simple


In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
# import dask, boto3
# import dask.dataframe as dd
import numpy as np
import json
import geopandas
import matplotlib.pyplot as plt
import shapely
# import kglab as kg


/home/fils/.local/lib/python3.10/site-packages/rdflib_jsonld/__init__.py:9: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  warnings.warn(
/home/fils/.local/lib/python3.10/site-packages/rdflib_jsonld/__init__.py:9: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  warnings.warn(


In [2]:
sparqlep = "http://graph.oceaninfohub.org/blazegraph/namespace/oih/sparql"

In [3]:
#@title
def get_sparql_dataframe(service, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

In [ ]:
rq_orgs = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?g ?s ?name
WHERE
{
    GRAPH ?g {
        ?s a <https://schema.org/Organization> .
        ?s <https://schema.org/name> ?name .
    }
    FILTER regex(str(?g), "^urn:gleaner.oih:orgs")
}
ORDER BY ?name

"""

In [4]:
rq_prov = """prefix prov: <http://www.w3.org/ns/prov#>
        PREFIX con: <http://www.ontotext.com/connectors/lucene#>
        PREFIX luc: <http://www.ontotext.com/owlim/lucene#>
        PREFIX con-inst: <http://www.ontotext.com/connectors/lucene/instance#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX schema: <https://schema.org/>
        PREFIX schemaold: <http://schema.org/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

       SELECT   ( COUNT(?hm) as ?count) ?wat  ?orgname ?domain
        WHERE
        {
           ?hm prov:wasAttributedTo ?wat .
           ?wat rdf:name ?orgname .
           ?wat rdfs:seeAlso ?domain
        }
        GROUP BY ?wat ?orgname ?domain
        ORDER BY DESC(?count)
        """

In [5]:
dfp = get_sparql_dataframe(sparqlep, rq_prov)
dfp['count'] = dfp["count"].astype(int) # convert count c to int
# dfp.set_index('orgname', inplace=True)


In [6]:
dfp

,count,wat,orgname,domain
0,8373,https://catalogue.odis.org/view/2993,EurOcean Vessels,https://infohub.eurocean.net/
1,8373,https://catalogue.odis.org/view/2993,EurOcean Projects,https://infohub.eurocean.net/
2,8373,https://catalogue.odis.org/view/2993,EurOcean Organizations,https://infohub.eurocean.net/
3,4726,https://oceaninfohub.org/.well-known/org/edmo,European Directory of Marine Organisations (ED...,https://edmo.seadatanet.org
4,4693,https://www.re3data.org/repository/seadatanet/...,EDMO SeaDataNet,https://edmo.seadatanet.org
5,4580,https://catalogue.odis.org/view/343,Ocean Biodiversity Information System,https://obis.org
6,4424,https://www.re3data.org/repository/obis,Ocean Biodiversity Information System,https://obis.org
7,3429,https://oceaninfohub.org/.well-known/org/edmerp,European Directory of Marine Environmental Res...,https://edmerp.seadatanet.org
8,3422,https://www.re3data.org/repository/seadatanet/...,EDMERP SeaDataNet,https://edmerp.seadatanet.org
9,1183,https://catalogue.odis.org/view/364,European Marine Observation and Data Network c...,https://emodnet.ec.europa.eu/


In [7]:
import pandas as pd
import requests

url = "https://raw.githubusercontent.com/iodepo/odis-arch/schema-dev-df/workflows/actions/sitemapcheck/test.csv"
r = requests.get(url)
open('temp.csv', 'wb').write(r.content)
df = pd.read_csv('temp.csv')

In [8]:
df.rename(columns={'propername':'orgname'}, inplace=True)

In [9]:
dfm = pd.merge(df, dfp, on='orgname', how='outer', left_index=False)
dfm = dfm.reset_index()


In [10]:
dfm.to_csv("sourcemerge.csv")

In [11]:
dfm

,index,name,orgname,code,description,url,type,count,wat,domain
0,0,africaioc,IOC Africa Data Portal,0.0,440 : africaioc VALID https://ioc-africa.org/s...,https://ioc-africa.org/sitemap.xml,sitemap,253.0,https://oceaninfohub.org/.well-known/org/afric...,https://ioc-africa.org
1,1,aquadocs,AquaDocs,0.0,aquadocs \t https://oih.aquadocs.org/aquadocs....,https://oih.aquadocs.org/aquadocs.json,sitegraph,1.0,http://hdl.handle.net/1834/41372,https://aquadocs.org
2,2,benguelacc,Benguela Current Convention (BCC) GeoData Portal,0.0,benguelacc \t https://raw.githubusercontent.co...,https://raw.githubusercontent.com/iodepo/odis-...,sitegraph,1.0,https://oceaninfohub.org/.well-known/org/bengu...,https://www.benguelacc.org/
3,3,caribbeanmarineatlas,Caribbean Marine Atlas catalogue,0.0,caribbeanmarineatlas \t https://raw.githubuser...,https://raw.githubusercontent.com/iodepo/odis-...,sitegraph,1.0,https://catalogue.odis.org/view/616,https://www.caribbeanmarineatlas.net/
4,4,cioos,CIOOS,0.0,507 : cioos VALID https://catalogue.cioos.ca/s...,https://catalogue.cioos.ca/sitemap/sitemap.xml,sitemap,498.0,https://oceaninfohub.org/.well-known/org/cioos,https://cioos.ca/
5,5,edmerp,European Directory of Marine Environmental Res...,0.0,3430 : edmerp VALID https://edmerp.seadatanet....,https://edmerp.seadatanet.org/sitemap.xml,sitemap,3429.0,https://oceaninfohub.org/.well-known/org/edmerp,https://edmerp.seadatanet.org
6,6,edmo,European Directory of Marine Organisations (ED...,0.0,4726 : edmo VALID https://edmo.seadatanet.org/...,https://edmo.seadatanet.org/sitemap.xml,sitemap,4726.0,https://oceaninfohub.org/.well-known/org/edmo,https://edmo.seadatanet.org
7,7,euroceanevents,EurOcean Events,1.0,ERROR euroceanevents : https://www.oceanexpert...,https://www.oceanexpert.org/assets/sitemaps/si...,sitemap,NaN,NaN,NaN
8,8,euroceanexperts,EurOcean Experts,1.0,ERROR euroceanexperts : https://www.oceanexper...,https://www.oceanexpert.org/assets/sitemaps/si...,sitemap,NaN,NaN,NaN
9,9,euroceaninstitutions,EurOcean Institutions,1.0,ERROR euroceaninstitutions : https://www.ocean...,https://www.oceanexpert.org/assets/sitemaps/si...,sitemap,NaN,NaN,NaN


,index,name,orgname,code,description,url,type,count,wat,domain
0,0,africaioc,IOC Africa Data Portal,0.0,440 : africaioc VALID https://ioc-africa.org/s...,https://ioc-africa.org/sitemap.xml,sitemap,253.0,https://oceaninfohub.org/.well-known/org/afric...,https://ioc-africa.org
1,1,aquadocs,AquaDocs,0.0,aquadocs \t https://oih.aquadocs.org/aquadocs....,https://oih.aquadocs.org/aquadocs.json,sitegraph,1.0,http://hdl.handle.net/1834/41372,https://aquadocs.org
2,2,benguelacc,Benguela Current Convention (BCC) GeoData Portal,0.0,benguelacc \t https://raw.githubusercontent.co...,https://raw.githubusercontent.com/iodepo/odis-...,sitegraph,1.0,https://oceaninfohub.org/.well-known/org/bengu...,https://www.benguelacc.org/
3,3,caribbeanmarineatlas,Caribbean Marine Atlas catalogue,0.0,caribbeanmarineatlas \t https://raw.githubuser...,https://raw.githubusercontent.com/iodepo/odis-...,sitegraph,1.0,https://catalogue.odis.org/view/616,https://www.caribbeanmarineatlas.net/
4,4,cioos,CIOOS,0.0,507 : cioos VALID https://catalogue.cioos.ca/s...,https://catalogue.cioos.ca/sitemap/sitemap.xml,sitemap,498.0,https://oceaninfohub.org/.well-known/org/cioos,https://cioos.ca/
5,5,edmerp,European Directory of Marine Environmental Res...,0.0,3430 : edmerp VALID https://edmerp.seadatanet....,https://edmerp.seadatanet.org/sitemap.xml,sitemap,3429.0,https://oceaninfohub.org/.well-known/org/edmerp,https://edmerp.seadatanet.org
6,6,edmo,European Directory of Marine Organisations (ED...,0.0,4726 : edmo VALID https://edmo.seadatanet.org/...,https://edmo.seadatanet.org/sitemap.xml,sitemap,4726.0,https://oceaninfohub.org/.well-known/org/edmo,https://edmo.seadatanet.org
7,7,euroceanevents,EurOcean Events,1.0,ERROR euroceanevents : https://www.oceanexpert...,https://www.oceanexpert.org/assets/sitemaps/si...,sitemap,NaN,NaN,NaN
8,8,euroceanexperts,EurOcean Experts,1.0,ERROR euroceanexperts : https://www.oceanexper...,https://www.oceanexpert.org/assets/sitemaps/si...,sitemap,NaN,NaN,NaN
9,9,euroceaninstitutions,EurOcean Institutions,1.0,ERROR euroceaninstitutions : https://www.ocean...,https://www.oceanexpert.org/assets/sitemaps/si...,sitemap,NaN,NaN,NaN
